In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
import os

/Users/martinbravodiaz/miniconda3/envs/imdb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    name='tmdb_fv',
    version=3,
)

2025-01-08 10:21:16,480 INFO: Initializing external client
2025-01-08 10:21:16,481 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 10:21:18,147 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [3]:
mr= project.get_model_registry()

retrieved_model=mr.get_model(
    name="movie_rating_model_2",
    version=1,
)

saved_model_dir=retrieved_model.download()

In [4]:
retrieved_xgboost_model=XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir+"/model.json")

In [5]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

movie_fg = fs.get_feature_group(
    name='tmdb_upcoming_movies',
    version=3,
)

df_new = movie_fg.read()

df_new

2025-01-08 10:21:23,836 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 10:21:23,842 INFO: Initializing external client
2025-01-08 10:21:23,842 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 10:21:25,126 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.61s) 


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year
0,1217624,The Evolution of Horror Cinema Worldwide,0.0,0.0,Post Production,2025-09-18 00:00:00+00:00,1000.0,90.0,1000.0,tt15679984,...,"Kaiti Wallen, Zoe Berriatúa, Rosmary Yaneva, D...",Jhon Jatenjor,None,Jhon Jatenjor,"David Conill, Ray Barlett",None,NaN,NaN,/q4GDeVerLjRNzlHfAFPaRoE28PF.jpg,2025
1,1266805,The Musicians,0.0,0.0,Post Production,2025-03-26 00:00:00+00:00,0.0,102.0,0.0,None,...,"Valérie Donzelli, Frédéric Pierrot, Mathieu Sp...",Grégory Magne,None,Grégory Magne,"Pierre-Louis Garnon, Frédéric Jouve",Grégoire Hetzel,NaN,NaN,None,2025
2,1315297,Dire Pines,0.0,0.0,Post Production,2025-01-24 00:00:00+00:00,0.0,0.0,3000.0,None,...,"Isaac Stacey, Matthew Trueman","Jim Couroux, Taylor Allison",Camille Poon,Jim Couroux,"Jim Couroux, Taylor Allison, David Daniels Won...",Jim Couroux,NaN,NaN,/mt2YmSyY6qz0VHrKsfATbd5YCPr.jpg,2025
3,931942,BLKNWS: Terms and Conditions,0.0,0.0,Post Production,2025-01-24 00:00:00+00:00,0.0,113.0,0.0,tt17497960,...,"Shaunette Renée Wilson, Peter Jay Fernandez, H...",Kahlil Joseph,Bradford Young,"Saidiya Hartman, Kahlil Joseph, Irvin Hunt","Kahlil Joseph, Onye Anyanwu, Nicolas Gonda, An...",None,NaN,NaN,/7wQgClzJTgzdKAqyNDfhzyt0NjR.jpg,2025
4,1405751,Coroner to the Stars,0.0,0.0,Post Production,2025-02-20 00:00:00+00:00,0.0,83.0,0.0,None,...,"Dr. Thomas Noguchi, Paul Guyet, Dan Medina, Dr...","Keita Ideno, Ben Hethcoat","Michelle Lawler, Erica Chan",None,"Chiaki Yanagimoto, John Henry Hinkel, Tommy An...","Mike Schanzlin, Mike Griffin",NaN,NaN,/9doStrDXyrhRpY2o78AugRcgJGP.jpg,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1389619,Parkside,0.0,0.0,Post Production,2025-01-15 00:00:00+00:00,0.0,0.0,150.0,tt32760279,...,"Mattaya Braun, Benjamin Jorgensen, Gillian Garvie",Clay Cesmistruk,None,Clay Cesmistruk,Clay Cesmistruk,None,NaN,NaN,None,2025
756,1235619,Les Bodin’s partent en vrille,0.0,0.0,Post Production,2025-03-19 00:00:00+00:00,0.0,0.0,0.0,tt32192665,...,"Vincent Dubois, Jean-Christophe Fraiscinet, Lé...",Frédéric Forestier,"Stéphane Le Parc, Christian Abomnes","Vincent Dubois, Jean-Christophe Fraiscinet","Claude Cyndecki, Frantz Richard",None,NaN,NaN,/4DUeoyYHwLIgzeCw8eszbjd0KgG.jpg,2025
757,1379877,Azaad,0.0,0.0,Post Production,2025-01-17 00:00:00+00:00,0.0,0.0,0.0,None,...,"Jiya Imran Amin, Amaan Devgn, Akshay Anand Koh...",Abhishek Kapoor,None,"K. Hariharan, Abhishek Kapoor, Ritesh Shah, Su...","Pragya Kapoor, Ronnie Screwvala",None,NaN,NaN,/yWFpiJcUJfXRbm3om48kNJ0MU7A.jpg,2025
758,1246595,El Set,0.0,0.0,Post Production,2025-06-01 00:00:00+00:00,0.0,0.0,0.0,None,...,"Nelly Karim, Mona Zaki, Ahmed Dawood, Tamer Na...",Marwan Hamed,Abdelsalam Moussa,Ahmed Mourad,None,None,NaN,NaN,None,2025


In [6]:
# Have budget
df_b = df_new[(df_new['budget'] > 10000000.0)]
df_b["imdb_votes"]=400

# We only consider movies that have a release date in the future
df_b["release_date"] = pd.to_datetime(df_b["release_date"])
df_b = df_b[df_b["release_date"] > datetime.now(timezone.utc)]

# fill runtime zero values with the mean of the non-zero values
runtime_without_zero = df_b[df_b["runtime"] != 0]
mean_runtime = runtime_without_zero["runtime"].mean()
df_b["runtime"] = df_b["runtime"].replace(0, mean_runtime)

df_b

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,132.0,75000000.0,tt10548174,...,"Ralph Fiennes, Joe Blakemore, Nathan Hall, Edv...",Danny Boyle,Anthony Dod Mantle,"Danny Boyle, Alex Garland","Danny Boyle, Peter Rice, Cillian Murphy, Berna...",None,NaN,400,/hVKHzr4GwSw0FepqhqQ0DDiYHNY.jpg,2025
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,132.0,85000000.0,tt27165187,...,"Jordan Alexa Davis, Anne Hathaway, Christian C...",David Robert Mitchell,Mike Gioulakis,David Robert Mitchell,"David Robert Mitchell, Chris Bender, J.J. Abra...",None,NaN,400,None,2026
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,132.0,68000000.0,tt17490712,...,"Desmond Chiam, Tati Gabrielle, Josh Lawson, Jo...",Simon McQuoid,Stephen F. Windon,Jeremy Slater,"Michael Clear, Simon McQuoid, Toby Emmerich, J...",Benjamin Wallfisch,NaN,400,/yN1bQ8XLh7Q2bEu3YBtp68vUeXl.jpg,2025
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,...,"George Angelopoulos, Maria Aliferi, Yorgo Cons...",Yiannis Stravolaimos,None,Yiannis Stravolaimos,"Ali Bakkioui, Yiannis Stravolaimos",None,NaN,400,/zY2vIC5q9waeSKDrYjAQnP6CyTz.jpg,2025
257,1127674,La città proibita,0.0,0.0,Post Production,2025-03-13 00:00:00+00:00,0.0,132.0,18218260.0,tt27766538,...,"Luca Zingaretti, Yaxi Liu, Sabrina Ferilli, Ma...",Gabriele Mainetti,Paolo Carnera,"Stefano Bises, Gabriele Mainetti, Davide Serino","Lorenzo Gangarossa, Gabriele Mainetti, Mario G...",None,NaN,400,None,2025
308,936075,Michael,0.0,0.0,Post Production,2025-10-01 00:00:00+00:00,0.0,132.0,155000000.0,tt11378946,...,"Rhyan Hill, KeiLyn Durrel Jones, Kevin Shinick...",Antoine Fuqua,Dion Beebe,John Logan,"David B. Householter, Graham King, John Branca...",None,NaN,400,/OJnvJXpRZZEfYj4L3WfEDQsaYQ.jpg,2025
316,1317288,Marty Supreme,0.0,0.0,Post Production,2025-12-25 00:00:00+00:00,0.0,132.0,70000000.0,tt32916440,...,"Timothée Chalamet, Gwyneth Paltrow, Odessa A'z...",Josh Safdie,Darius Khondji,"Ronald Bronstein, Josh Safdie","Eli Bush, Sara Rossein, Ronald Bronstein, Timo...",None,NaN,400,None,2025
416,1249289,Alarum,0.0,0.0,Post Production,2025-01-16 00:00:00+00:00,0.0,95.0,20000000.0,tt31456973,...,"Beau Bommarito, La Monde Byrd, Anton Narinskiy...",Michael Polish,None,Alexander Vesha,"Luke Taylor, Matthew Helderman, Joel Cohen, Gw...",None,NaN,400,/luIf8RCY8sKyluZUJ6bzwqhYC0x.jpg,2025
461,680493,Return to Silent Hill,0.0,0.0,Post Production,2025-04-11 00:00:00+00:00,0.0,132.0,40000000.0,tt22868010,...,"Melissa Graham, Slaviša Ivanović, Robert Stran...",Christophe Gans,Pablo Rosso,"William Josef Schneider, Sandra Vo-Anh, Christ...","Joe Jenckes, Jay Taylor, David M. Wulf, Jonath...",Akira Yamaoka,NaN,400,/yFNDmvwVUiMrrtLGjnMNTIsW91z.jpg,2025
477,83533,Avatar: Fire and Ash,0.0,0.0,Post Production,2025-12-17 00:00:00+00:00,0.0,132.0,400000000.0,tt1757678,...,"Cliff Curtis, CCH Pounder, Giovanni Ribisi, Tr...",James Cameron,Russell Carpenter,"James Cameron, Rick Jaffa, Shane Salerno, Aman...","Peter M. Tobyansen, Jon Landau, James Cameron",Simon Franglen,NaN,400,/vDdFxksDMkUJjZXd00OKJfwXKre.jpg,2025


In [7]:
from utils import new_features

df1= new_features(df_b)

In [8]:
df1.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,release_year,first_producer,first_actor,first_company
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,132.0,75000000.0,tt10548174,...,"Danny Boyle, Alex Garland","Danny Boyle, Peter Rice, Cillian Murphy, Berna...",None,NaN,400,/hVKHzr4GwSw0FepqhqQ0DDiYHNY.jpg,2025,3,16,4
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,132.0,85000000.0,tt27165187,...,David Robert Mitchell,"David Robert Mitchell, Chris Bender, J.J. Abra...",None,NaN,400,None,2026,6,9,8
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,132.0,68000000.0,tt17490712,...,Jeremy Slater,"Michael Clear, Simon McQuoid, Toby Emmerich, J...",Benjamin Wallfisch,NaN,400,/yN1bQ8XLh7Q2bEu3YBtp68vUeXl.jpg,2025,16,4,11
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,...,Yiannis Stravolaimos,"Ali Bakkioui, Yiannis Stravolaimos",None,NaN,400,/zY2vIC5q9waeSKDrYjAQnP6CyTz.jpg,2025,0,6,17
257,1127674,La città proibita,0.0,0.0,Post Production,2025-03-13 00:00:00+00:00,0.0,132.0,18218260.0,tt27766538,...,"Stefano Bises, Gabriele Mainetti, Davide Serino","Lorenzo Gangarossa, Gabriele Mainetti, Mario G...",None,NaN,400,None,2025,14,11,16


In [9]:
df_encoded = df1.copy()
df_encoded = df_encoded[["runtime", "release_year", "imdb_votes", "budget", "genres", "first_producer", "first_actor", "first_company", "original_language"]]
df_encoded['genres'] = df_encoded['genres'].str.split(', ')  # Split by comma and space
genres_encoded = df_encoded['genres'].explode()  # Flatten the list
one_hot = pd.get_dummies(genres_encoded).groupby(level=0).sum()

one_hot.columns = one_hot.columns.str.lower()  # Lowercase column names
one_hot.columns = one_hot.columns.str.replace(' ', '_')  # Replace spaces with underscores

# Add missing columns if not already in the DataFrame
all_genres=['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']
for col in all_genres:
        if col not in one_hot.columns:
            one_hot[col] = 0  # or df_encoded[col] = False if they were boolean flags

# Merge one-hot encoding with the original DataFrame
train_features = pd.concat([df_encoded, one_hot], axis=1)
train_features=train_features.drop(["genres"], axis=1)
train_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 57 to 745
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   runtime            20 non-null     float64
 1   release_year       20 non-null     int32  
 2   imdb_votes         20 non-null     int64  
 3   budget             20 non-null     float64
 4   first_producer     20 non-null     int64  
 5   first_actor        20 non-null     int64  
 6   first_company      20 non-null     int64  
 7   original_language  20 non-null     int64  
 8   action             20 non-null     int64  
 9   adventure          20 non-null     int64  
 10  comedy             20 non-null     int64  
 11  crime              20 non-null     int64  
 12  drama              20 non-null     int64  
 13  family             20 non-null     int64  
 14  fantasy            20 non-null     int64  
 15  history            20 non-null     int64  
 16  horror             20 non-null 

In [10]:
# Ensure the column order matches the expected order
expected_columns = ['release_year', 'runtime', 'budget', 'original_language', 'first_company', 'first_producer', 'first_actor', 'imdb_votes', 'action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']
train_features=train_features[expected_columns]

train_features

,release_year,runtime,budget,original_language,first_company,first_producer,first_actor,imdb_votes,action,adventure,...,history,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western
57,2025,132.0,75000000.0,0,4,3,16,400,0,0,...,0,1,0,0,0,1,0,0,0,0
60,2026,132.0,85000000.0,0,8,6,9,400,0,1,...,0,0,0,0,0,0,0,1,0,0
108,2025,132.0,68000000.0,0,11,16,4,400,1,1,...,0,0,0,0,0,0,0,0,0,0
198,2025,196.0,22000000.0,0,17,0,6,400,0,0,...,0,0,0,0,0,0,0,0,0,0
257,2025,132.0,18218260.0,3,16,14,11,400,1,0,...,0,0,0,0,0,0,0,0,0,0
308,2025,132.0,155000000.0,0,10,4,18,400,0,0,...,1,0,1,0,0,0,0,0,0,0
316,2025,132.0,70000000.0,0,1,7,19,400,0,0,...,0,0,0,0,0,0,0,0,0,0
416,2025,95.0,20000000.0,0,3,15,1,400,1,0,...,0,0,0,0,0,0,0,1,0,0
461,2025,132.0,40000000.0,0,9,12,15,400,0,0,...,0,1,0,1,0,0,0,0,0,0
477,2025,132.0,400000000.0,0,0,18,3,400,0,1,...,0,0,0,0,0,1,0,0,0,0


In [11]:
predicted = retrieved_xgboost_model.predict(train_features)

In [12]:
train_features["predicted_rating"]=predicted

train_features

,release_year,runtime,budget,original_language,first_company,first_producer,first_actor,imdb_votes,action,adventure,...,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western,predicted_rating
57,2025,132.0,75000000.0,0,4,3,16,400,0,0,...,1,0,0,0,1,0,0,0,0,3.845231
60,2026,132.0,85000000.0,0,8,6,9,400,0,1,...,0,0,0,0,0,0,1,0,0,5.373774
108,2025,132.0,68000000.0,0,11,16,4,400,1,1,...,0,0,0,0,0,0,0,0,0,5.110221
198,2025,196.0,22000000.0,0,17,0,6,400,0,0,...,0,0,0,0,0,0,0,0,0,7.384965
257,2025,132.0,18218260.0,3,16,14,11,400,1,0,...,0,0,0,0,0,0,0,0,0,5.168054
308,2025,132.0,155000000.0,0,10,4,18,400,0,0,...,0,1,0,0,0,0,0,0,0,6.168319
316,2025,132.0,70000000.0,0,1,7,19,400,0,0,...,0,0,0,0,0,0,0,0,0,6.237243
416,2025,95.0,20000000.0,0,3,15,1,400,1,0,...,0,0,0,0,0,0,1,0,0,4.569636
461,2025,132.0,40000000.0,0,9,12,15,400,0,0,...,1,0,1,0,0,0,0,0,0,5.308178
477,2025,132.0,400000000.0,0,0,18,3,400,0,1,...,0,0,0,0,1,0,0,0,0,4.606920


In [13]:
df_rated = pd.concat([df_b, train_features], axis=1)

df_rated

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,horror,music,mystery,romance,science_fiction,tv_movie,thriller,war,western,predicted_rating
57,1100988,28 Years Later,0.0,0.0,Post Production,2025-06-16 00:00:00+00:00,0.0,132.0,75000000.0,tt10548174,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.845231
60,1101383,Flowervale Street,0.0,0.0,Post Production,2026-03-12 00:00:00+00:00,0.0,132.0,85000000.0,tt27165187,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.373774
108,931285,Mortal Kombat 2,0.0,0.0,Post Production,2025-10-23 00:00:00+00:00,0.0,132.0,68000000.0,tt17490712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.110221
198,1277016,The Queen of Heaven,0.0,0.0,Post Production,2025-04-04 00:00:00+00:00,0.0,196.0,22000000.0,tt25049058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.384965
257,1127674,La città proibita,0.0,0.0,Post Production,2025-03-13 00:00:00+00:00,0.0,132.0,18218260.0,tt27766538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.168054
308,936075,Michael,0.0,0.0,Post Production,2025-10-01 00:00:00+00:00,0.0,132.0,155000000.0,tt11378946,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.168319
316,1317288,Marty Supreme,0.0,0.0,Post Production,2025-12-25 00:00:00+00:00,0.0,132.0,70000000.0,tt32916440,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.237243
416,1249289,Alarum,0.0,0.0,Post Production,2025-01-16 00:00:00+00:00,0.0,95.0,20000000.0,tt31456973,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.569636
461,680493,Return to Silent Hill,0.0,0.0,Post Production,2025-04-11 00:00:00+00:00,0.0,132.0,40000000.0,tt22868010,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.308178
477,83533,Avatar: Fire and Ash,0.0,0.0,Post Production,2025-12-17 00:00:00+00:00,0.0,132.0,400000000.0,tt1757678,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.606920


In [14]:
df_rated.to_csv("movies.csv", index=False,sep=";")